<a href="https://colab.research.google.com/github/kchenTTP/python-series/blob/main/advanced_python_functions/Advanced_Python_Functions_Part_1_Higher_Order_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Advanced Python Functions Part 1: Higher Order Functions**

In our intermediate class, we covered default parameters, variable-length arguments, function scopes, and function documentation. Now, let's take it further by exploring some more advanced function concepts.

## **Table of Contents**

- [Higher-Order Functions](#scrollTo=XR0znysK2u90)
- [Lambda Functions](#scrollTo=-F-K0eG5xRLz)
- [Common Higher-Order Functions](#scrollTo=9iKeKO2i7lrA)
  - [`sorted()`](#scrollTo=Kfk5UEIm2N9Z)
  - [`map()`](#scrollTo=O-ic2tDwRbxX)
  - [`filter()`](#scrollTo=7Qd5Mz57UF9z)


In [1]:
from typing import Callable, Any

## **Higher-Order Functions**

Higher-order functions are functions that can either:

- Accept other functions as arguments
- Return functions as results
- Perform both of the above


### **Example 1: Function Tester**

This function takes another function as input and tests it against a given value. If the function's return value matches the `expected_result`, the test passes; otherwise, it fails.


In [2]:
def tester(func: Callable, *, expected_result: Any, **func_args) -> None:
  """Test if function will return expected results"""
  try:
    res = func(**func_args)
    if not res and expected_result:
      print(f"FAILED: <{func.__name__}> expected '{expected_result}', got '{res}' instead")
      return
    assert res == expected_result
    print(f"PASSED: <{func.__name__}>")
  except AssertionError:
    print(f"FAILED: <{func.__name__}> expected '{expected_result}', got '{res}' instead")

In [3]:
def greet(name: str, greeting: str = "Hello") -> str:
  return f"{greeting}, {name}!"

def do_sth() -> None:
  """Function that is incorrectly defined"""
  pass


tester(greet, expected_result="Hello, Mike!", name="Mike")
tester(greet, expected_result="G'day, Tim!", name="Tim", greeting="G'day")
tester(greet, expected_result="Hey, Mike!", name="Mike")
tester(do_sth, expected_result=["a", "b", "c"])

PASSED: <greet>
PASSED: <greet>
FAILED: <greet> expected 'Hey, Mike!', got 'Hello, Mike!' instead
FAILED: <do_sth> expected '['a', 'b', 'c']', got 'None' instead


### **Example 2: Create Logger**

This function creates custom logging functions with flexible log levels and timezone options to generate time-stamped log messages with a consistent format.

<br>

**Components**

- **Parameters**
  - *`log_level` (str):* Specifies the log level or prefix (e.g., "INFO", "WARNING").
  - *`timezone` (str):* An optional parameter with a default value of "America/New_York" that sets the timezone for the timestamp using `pytz`.
  - *`cache_len` (list):* A mutable list that stores the length of the longest `log_level`. This ensures all log entries align properly by standardizing the width of the log level display. A list of one integer can be provided to modify the prefix width but it needs to be larger than the length of `log_level`.

- **Returns**
  - A logger function, which takes a single `message` parameter (the log message) and prints it with a formatted timestamp and log level prefix.

<br>

> 📒 NOTE: Mutable default arguments (like the empty list in the function above) can be risky due to unintended side effects. However, in this case, I'm intentionally using a mutable default to track the length of different logging texts across the program.


In [5]:
from datetime import datetime
import pytz # lib to make sure your code uses the correct timezone you're in

def create_logger(log_level: str, timezone: str="America/New_York", cache_len: list=[]) -> Callable[[str], None]:
  length = len(log_level)
  if not cache_len:
    cache_len.append(length)
  else:
    if length > cache_len[0]:
      cache_len[0] = length

  def logger(message: str) -> None:
    current_time = datetime.strftime(datetime.now(tz=pytz.timezone(timezone)), "%Y-%m-%d %H:%M:%S")
    print(f"{log_level:<{cache_len[0]}} - {current_time} - {message}")

  return logger


info_logger = create_logger("INFO")
warning_logger = create_logger("WARNING")

info_logger("This is an info message")
warning_logger("This is a warning message")

INFO    - 2025-04-04 14:53:49 - This is an info message
WARNING - 2025-04-04 14:53:49 - This is a warning message


**Explanation**

1. Updating `cache_len` for Alignment:

  `cache_len` is initialized as an empty list, storing the longest `log_level` length found so far. Using a mutable default argument allows each call to `create_logger` to share this list and alignment using the proper character length.

  - If `cache_len` is empty, it initializes with the current `log_level` length.
  - If `cache_len` already has a value, it updates only if the new `log_level` length is greater than the stored length.

2. Defining the `logger` Inner Function:

  This inner function `logger` formats and prints log messages using:
  
  - `log_level`, left-aligned according to `cache_len[0]` to ensure consistent width across different loggers.
  - `current_time`, a timestamp based on the `timezone` argument.
  - `message`, the message to be logged.

3. Returning the `logger` Function:

  The `create_logger` function returns this `logger` function, which can be used as a customized logger with a prefix.


## **Lambda Functions**

Higher-order functions are powerful because they allow you to treat functions as first-class citizens, making your code more flexible and reusable.

However, sometimes you only need a function once, and defining a full function for that can feel unnecessary. That's where **lambda functions** come in. They let you create quick, one-off functions without the overhead of a formal function definition.

Lambda functions are small, anonymous functions that can take any number of arguments but must contain a **single expression**. They are particularly useful for short operations where defining a full function would be excessive.

<br>

**Key Concepts**

- **Single Expression Only**

  A lambda function can only contain one expression, which is evaluated and returned.

- **Can be used as function arguments**

  Lambda functions can be passed as arguments to higher-order functions like `map()`, `filter()`, and `sorted()`.
  
- **Useful for sorting and filtering**

  They are often used for single usage operations like sorting lists or filtering data without needing a named function.

- **Should be simple and readable**

  Lambda functions are best for simple operations; complex logic should be implemented in a regular function for clarity.

<br>

**Syntax**

```python
lambda arguments: expression
```


To show how lambda functions truely work underneath the hood. Here's a few lambda functions and their equivalent functions if they were defined like a regular function:

In [ ]:
def square_verbose(x):
  return x ** 2

square = lambda x: x ** 2

print(square_verbose(3))
print(square(3))

9
9


In [15]:
# Lambda with default arguments
def say_hi_verbose(name, greeting="Hello"):
  return f"{greeting}, {name}!"

say_hi = lambda name, greeting="Hello": f"{greeting}, {name}!"

print(say_hi_verbose("Mike"))
print(say_hi("Mike"))

Hello, Mike!
Hello, Mike!


In [16]:
# Lambda with conditional logic
is_adult = lambda age: "Adult" if age >= 18 else "Minor"
print(is_adult(25))
print(is_adult(15))

Adult
Minor


In [17]:
# Lambda with list comprehension
def format_names_verbose(lst):
  res = []
  for s in lst:
    res.append(str(s).title())
  return res

format_names = lambda lst: [str(x).title() for x in lst]

print(format_names_verbose(["alice", "BOB", "ChArLiE"]))
print(format_names(["alice", "BOB", "ChArLiE"]))

def flatten_list_verbose(lst):
  res = []
  for sublist in lst:
    for item in sublist:
      res.append(item)
  return res

flatten_list = lambda lst: [item for sublist in lst for item in sublist]
print(flatten_list_verbose([[1, 2, 3], [4, 5], [6, 7, 8, 9]]))
print(flatten_list([[1, 2, 3], [4, 5], [6, 7, 8, 9]]))

['Alice', 'Bob', 'Charlie']
['Alice', 'Bob', 'Charlie']
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


> ❗**IMPORTANT:** You typically don't assign a lambda function to a variable (this is highly discouraged). Instead, use it directly where an anonymous function is needed, such as [higher-order functions](#scrollTo=XR0znysK2u90), which we will cover some in the next section.


## **Common Higher-Order Functions in Python**

- `sorted()`
- `map()`
- `filter()`
- `reduce()`

> 💡 These functions are very often used with [lambda functions](#scrollTo=-F-K0eG5xRLz)


### `sorted()`

The `sorted()` function takes a list and sorts it based on the criteria specified in the `key` argument.


In [ ]:
# How sorted works
l1 = [2, 5, 1, 3, 4]

print(sorted(l1))
print(sorted(l1, key=lambda x: x)) # the is the default lambda function when sorting a list
print(sorted(l1, reverse=True))

# Sort list based on index 0
l2 = [[4, 7], [9, 1], [3, 2]]

print(sorted(l2, key=lambda x: x[0]))

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[5, 4, 3, 2, 1]
[[3, 2], [4, 7], [9, 1]]


We can even sort a list of dictionaries:


In [19]:
# Sort dictionary basd on a dictionary key
def sort_objects(
    objects: list[dict[str, Any]],
    sort_key: str,
    default_value: Any = "",
    reverse: bool = False
) -> list[dict]:
  """Sort a list of dictionaries by a single key.

  Args:
    objects (list[dict[str, Any]]): List of dictionaries to sort
    sort_key (str): The dictionary key to sort by
    reverse (bool): Whether to reverse the sort

  Returns:
    Sorted list of dictionaries
  """
  return sorted(
      objects,
      key=lambda x: x.get(sort_key, default_value), # This will return the value of the sort_key which the sorted function will sort base on the values
      reverse=reverse
  )


data = [
    {"name": "Timmy", "age": 35, "salary": 60000},
    {"name": "John", "age": 25, "salary": 55000},
    {"name": "Andrea", "age": 30, "salary": 75000}
]

print("Sort by age:", sort_objects(data, "age"))
print("Sort by name:", sort_objects(data, "name"))
print("Sort by salary (rev):", sort_objects(data, "salary", reverse=True)) # Sort by salary (descending)

Sort by age: [{'name': 'John', 'age': 25, 'salary': 55000}, {'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}]
Sort by name: [{'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'John', 'age': 25, 'salary': 55000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}]
Sort by salary (rev): [{'name': 'Andrea', 'age': 30, 'salary': 75000}, {'name': 'Timmy', 'age': 35, 'salary': 60000}, {'name': 'John', 'age': 25, 'salary': 55000}]


### **`map()`**

Applies a function to every item in an iterable, creating a new iterable with the results. Useful for transforming data without using loops.


In [20]:
# Squaring a list of numbers
numbers = [1, 2, 3, 4, 5]
squared_numbers = list(map(lambda x: x ** 2, numbers))
print(squared_numbers)

[1, 4, 9, 16, 25]


In [21]:
# Process a list of names
names = ["Janice", "Allen  ", "\nBarba"]
processed_names = list(map(lambda x: x.strip().upper(), names))
print(processed_names)

['JANICE', 'ALLEN', 'BARBA']


📒 **NOTE:**

Even though `map()` is powerful, list comprehensions are often preferred. The code above can be rewritten using a list comprehension like this:


In [22]:
numbers = [1, 2, 3, 4, 5]
squared_numbers = [x ** 2 for x in numbers]
print(squared_numbers)

[1, 4, 9, 16, 25]


In [23]:
names = ["Janice", "Allen  ", "\nBarba"]
processed_names = [x.strip().upper() for x in names]
print(processed_names)

['JANICE', 'ALLEN', 'BARBA']


### **`filter()`**

Selects items from an iterable based on a function that returns `True` or `False`, returning only items that meet the condition.


In [24]:
# Filter even numbers
numbers = [1, 2, 3, 4, 5]
even_numbers = list(filter(lambda x: x % 2 == 0, numbers))
print(even_numbers)

# Filter number greater than or equal to 3
filtered_numbers = list(filter(lambda x: x >= 3, numbers))
print(filtered_numbers)

[2, 4]
[3, 4, 5]


Here we have a group of people, let's use filter to see if they attended the event and whether they're late:


In [34]:
attendees = [
    {"name": "Andy", "attended": True, "time_arrived": "11:34"},
    {"name": "Joclyn", "attended": True, "time_arrived": "12:00"},
    {"name": "Chandler", "attended": True, "time_arrived": "12:01"},
    {"name": "David", "attended": False, "time_arrived": ""}
]

In [32]:
# Get people who attended an event
def attended_event(attendees: list[dict[str, Any]]) -> list[dict[str, str]]:
  attended = list(filter(lambda x: x["attended"], attendees))
  return [p["name"] for p in attended]

present_attendees = attended_event(attendees)
print(present_attendees)

['Andy', 'Joclyn', 'Chandler']


In [35]:
# Get people who are late
def is_late(attendees: list[dict[str, Any]], scheduled_time: str) -> list[dict[str, str]]:
  attended = list(filter(lambda x: x["attended"], attendees))

  hr, min = scheduled_time.split(":")
  arrived_same_hr = list(filter(lambda x: int(x["time_arrived"].split(":")[0]) >= int(hr), attended))
  late = list(filter(lambda x: int(x["time_arrived"].split(":")[1]) > int(min), arrived_same_hr))

  return [p["name"] for p in late]


late_attendees = is_late(attendees, scheduled_time="12:00")
print(late_attendees)

['Chandler']


### **`reduce()` (Additional Material)**

📒 NOTE: This section covers `reduce()`. While it is a very powerful function, it is less commonly used and can be harder to grasp. Feel free to skip this section until you're comfortable with lambda functions and passing functions as arguments.

The `reduce()` function successively reduces an iterable to a single cumulative value by applying a specified function to pairs of items. The function used with reduce must accept two positional arguments, as it operates on pairs of items in sequence.

It is commonly used for accumulating data, such as summing, multiplying, or otherwise combining items.

> `reduce()` requires importing from the functools module.

**Syntax**

```python
reduce(lambda arg1, arg2: expression, Iterable)
```


In [36]:
from functools import reduce

In [37]:
# Multiply all the numbers in the list
numbers = [2, 3, 2, 5]
product = reduce(lambda prev, next: prev * next, numbers)
print(product)

60


Here's how the cell above works:

1. `reduce` applies a function cumulatively to items in an iterable, from left to right, to reduce them to a single value.
2. Lambda Function: The `lambda x, y: x * y` function takes two parameters, `x` and `y`, and multiplies them. This function will be applied to each pair of numbers in numbers in sequence. `x` is the result of the lambda function for the previous pair.
3. Reducing the List:
  - `reduce` begins with the first two items in numbers (2 and 3) and multiplies them to get 6.
  - Then it takes 6 and the next item (2) and multiplies them, resulting in 12.
  - Finally, it takes 12 and the last item (5), multiplying them to get 60.


Here's a couple more examples:


In [41]:
# Find maximum value
numbers = [3, 5, 2, 8, 1]
max_value = reduce(lambda x, y: max(x, y), numbers) # equivilant of reduce(max, numbers)
print(max_value)

8


In [42]:
# Merge multiple dictionaries with conflict resolution
dicts = [
    {'a': 1, 'b': 2},
    {'b': 3, 'c': 4},
    {'c': 5, 'd': 6}
]

def merge_resolve(acc: dict, curr: dict) -> dict:
  # Unpack previous dictionary to local dictionary d
  d = {**acc}

  # If conflicted keys, choose the larger value
  for k, v in curr.items():
    if k in d:
      d[k] = max(d[k], v)
    else:
      d[k] = v

  return d

merged = reduce(merge_resolve, dicts)
print(merged)

{'a': 1, 'b': 3, 'c': 5, 'd': 6}


In [44]:
# @title Advanced Usage of `reduce`

# Compose multiple functions into a pipeline that takes a single argument
def compose_functions(*funcs) -> Callable:
  """Compose (nest) multiple functions into a single function."""
  return reduce(lambda f, g: lambda x: f(g(x)), funcs)

to_upper = lambda s: s.upper()
remove_spaces = lambda s: s.replace(' ', '_')
add_exclamation = lambda s: s + '!'

# Here we use compose_function to basically create a string processing pipeline
pipeline = compose_functions(add_exclamation, to_upper, remove_spaces)
print(pipeline("a quick brown fox jumps over a box"))

# Here's a url processing pipeline
process_url = compose_functions(
    lambda s: s.strip('/'),
    lambda s: s.replace('http', 'https'),
    lambda s: s.lower()
)
print(process_url("HTTP://WWW.GOOGLE.COM/"))

A_QUICK_BROWN_FOX_JUMPS_OVER_A_BOX!
https://www.google.com


Here's the verbose version of the above `compose_functions()` example:


In [46]:
def verbose_pipeline(*funcs):
  def compose(f, g):
    def composed_function(x):
      g_result = g(x)
      print(f"After {g.__name__}: {g_result}")
      f_result = f(g_result)
      print(f"After {f.__name__}: {f_result}")
      return f_result
    return composed_function

  return reduce(compose, funcs)

# Make our functions named for better debugging
def remove_spaces(s):
  return s.replace(' ', '')

def to_upper(s):
  return s.upper()

def add_exclamation(s):
  return s + '!'

pipeline = verbose_pipeline(add_exclamation, to_upper, remove_spaces)
print(pipeline("a quick brown fox jumps over a box"))

print("=====")
process_url = verbose_pipeline(
    lambda s: s.strip('/'),
    lambda s: s.replace('http', 'https'),
    lambda s: s.lower()
)
print(process_url("HTTP://WWW.GOOGLE.COM/"))

After remove_spaces: aquickbrownfoxjumpsoverabox
After to_upper: AQUICKBROWNFOXJUMPSOVERABOX
After add_exclamation: AQUICKBROWNFOXJUMPSOVERABOX!
After composed_function: AQUICKBROWNFOXJUMPSOVERABOX!
AQUICKBROWNFOXJUMPSOVERABOX!
=====
After <lambda>: http://www.google.com/
After <lambda>: https://www.google.com/
After <lambda>: https://www.google.com
After composed_function: https://www.google.com
https://www.google.com


> 🤖 Higher-order functions can get complicated quickly. It's okay if you don't understand them right away. Take your time and make sure you fully grasp them before moving on.


## **Conclusion**

That concludes our introduction to higher-order functions! We've explored how functions can be passed as arguments, the power of lambda functions, and common higher-order functions like `sorted()`, `map()`, and `filter()`.

For a deeper look at these concepts, check out this article on higher-order functions and closures: [First-Class Functions and Closures in Python - freeCodeCamp](https://www.freecodecamp.org/news/first-class-functions-and-closures-in-python/)

<br>

In the next class, we'll take things a step further by looking at higher-order functions that return functions as outputs. We'll dive into `closures` and `decorators`, two powerful concepts that enable more advanced function manipulation and customization. See you then! 🤠
